### List of all planets in the exoplanets archive less than 70pc

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np


r = pd.read_csv('PS_2023.04.20_16.50.21.csv', comment='#')
r['st_spectype'].loc[0:4] = 'M5.5'
r['st_teff'].loc[0:4] = '3300'
r['pl_msinie'].loc[0:4] = 1.2
r['pl_orbsmax'].loc[0:4] = 0.049

# Remove duplicate entries for same planet:
pl = np.unique(r['pl_name'])
ind = [np.where(r['pl_name'] == pl[i])[0][0] for i in range(len(pl))]
p = r.loc[ind]
p = p.reset_index(drop = True)

p

,loc_rowid,pl_name,hostname,hd_name,hip_name,gaia_id,default_flag,sy_snum,sy_pnum,discoverymethod,...,sy_imagerr2,sy_zmag,sy_zmagerr1,sy_zmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,rowupdate,pl_pubdate,releasedate
0,8,14 Her b,14 Her,HD 145675,HIP 79248,Gaia DR2 1385293808145621504,0,1,2,Radial Velocity,...,NaN,NaN,NaN,NaN,6.38300,0.000351,-0.000351,2018-04-25,2004-01,2014-08-21
1,16,16 Cyg B b,16 Cyg B,HD 186427,HIP 96901,Gaia DR2 2135550755683407232,1,3,1,Radial Velocity,...,NaN,NaN,NaN,NaN,6.06428,0.000603,-0.000603,2018-09-04,2017-03,2018-09-06
2,34,2MASS J01225093-2439505 b,2MASS J01225093-2439505,NaN,NaN,Gaia DR2 5040416186560252416,0,1,1,Imaging,...,NaN,NaN,NaN,NaN,12.90250,0.001151,-0.001151,2015-09-02,2015-05,2015-09-03
3,38,2MASS J12073346-3932539 b,2MASS J12073346-3932539,NaN,NaN,Gaia DR2 3459372646830687104,1,1,1,Imaging,...,NaN,NaN,NaN,NaN,17.40760,0.002154,-0.002154,2016-02-24,2004-10,2016-02-25
4,42,2MASS J22362452+4751425 b,2MASS J22362452+4751425,NaN,NaN,Gaia DR2 1986772848034988544,1,1,1,Imaging,...,NaN,NaN,NaN,NaN,11.94840,0.000737,-0.000737,2016-11-09,2017-01,2016-11-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,34202,ups And b,ups And,HD 9826,HIP 7513,Gaia DR2 348020448377061376,0,2,3,Radial Velocity,...,NaN,NaN,NaN,NaN,3.98687,0.008937,-0.008937,2016-11-09,1997-01,2016-11-10
975,34209,ups And c,ups And,HD 9826,HIP 7513,Gaia DR2 348020448377061376,0,2,3,Radial Velocity,...,NaN,NaN,NaN,NaN,3.98687,0.008937,-0.008937,2021-08-03,2010-06,2021-08-03
976,34215,ups And d,ups And,HD 9826,HIP 7513,Gaia DR2 348020448377061376,0,2,3,Radial Velocity,...,NaN,NaN,NaN,NaN,3.98687,0.008937,-0.008937,2021-09-20,2021-05,2021-09-20
977,34221,ups Leo b,ups Leo,NaN,NaN,Gaia DR2 3794167001116433152,1,1,1,Radial Velocity,...,NaN,NaN,NaN,NaN,4.03040,0.008513,-0.008513,2022-01-10,2021-12,2022-01-10


## Radius:

In [2]:
def PlanetMass2Radius(M):
    ''' Theoretical mass-radius relation for planets and brown dwarfs by Jared
        taken from 
        https://jaredmales.github.io/mxlib-doc/group__planets.html#ga4b350ecfdeaca1bedb897db770b09789
    '''
    try:
        M = M.to(u.Mearth)
        M = M.value
    except:
        pass
    
    if M < 4.1:
        R = M**(1/3)
        
    if M >= 4.1 and M < 15.84:
        R = 0.62 * M**(0.67)
        
    if M >= 15.84 and M < 3591.1:
        coeff = [14.0211, -44.8414, 53.6554, -25.3289, 5.4920, -0.4586]
        power = [0, 1, 2, 3, 4, 5]
        R = 0
        for i in range(6):
            R += coeff[i] * (np.log10(M)**power[i])
            
    if M >= 3591.1:
        R = 32.03 * M**(-1/8)
        
    return R

def PlanetRadius2Mass(planet_radius):
    from myastrotools.tools import PlanetMass2Radius
    Ms = np.logspace(-0.5,3,1000)
    Rs = []
    for M in Ms:
        Rs.append(PlanetMass2Radius(M))
    from scipy.interpolate import interp1d
    f = interp1d(Rs,Ms)
    return f(planet_radius)

In [3]:
# If radius not present in archive, use MR relation to estimate radius:
p['M2R infered radius [Rearth]'] = np.nan
for i in range(len(p)):
    if np.isnan(p.loc[i]['pl_rade']):
        if not np.isnan(p.loc[i]['pl_msinie']):
            p['M2R infered radius [Rearth]'][i] = PlanetMass2Radius(p.loc[i]['pl_msinie'])
            
p['M2R infered radius [Rearth]']

0      11.981839
1      11.686000
2            NaN
3            NaN
4            NaN
         ...    
974    10.823049
975    11.739158
976    11.976042
977    10.414397
978    11.754746
Name: M2R infered radius [Rearth], Length: 979, dtype: float64

## Planet-Star separation:
Semi-major axis given in archive:

In [4]:
print([p.loc[i]['pl_orbsmax'] for i in range(len(p))])

[2.8, 1.66, nan, 55.0, 230.0, 0.995, 2.1, 3.6, 11.6, 13.2, 0.05235, 0.1134, 0.24, 5.84, nan, 0.792, 2.11, 0.04956, 0.2175, 0.476, 1.9, 2.153, 0.484, 0.7, 260.0, 7.99, nan, 0.1101, 0.777, 0.83, 0.78, 1662.0, 2.06, 1.48, 0.0428, 0.435, 0.06, 0.68, 2.36, 0.3, 1.34, 0.0185, 7506.0, 5.75, 0.1462, 0.0733, 0.0422, 0.0651, 0.0662, nan, 0.02279, 0.0525, nan, 0.0457, 0.0738, 0.021, 0.035, 0.054, nan, 0.0476, 0.163506, 0.912, 0.5714, nan, nan, 0.026, nan, 0.072, 5.4, 0.053, 0.0607, 0.1254, 1.0304, 2.41, 0.092, 0.129, 0.309, 0.034411, 16.763449, 0.283, 0.384214, 0.97, 0.0818, 0.101, 0.091101, 0.036467, 0.49, 0.079, nan, 0.0197, 0.057, 0.698, 0.95, 5.78, 0.091179, 4.5, 0.1434, 0.364, 0.19394, 0.08208, 0.03282, 0.1264, 0.089, 0.142, 0.03557, 0.01589, 0.0876, 0.338, 1.292, 0.035, 0.061, 0.204, 0.0287, nan, 0.026, 0.0252, 0.081, 0.05402, 0.0608, 0.029, 0.089, 0.0789, 0.2324, 1.4, 0.111, 0.082, 0.06, 3.6, 0.178, 0.0308, 3.53, 0.068, 0.01734, 43.5, 0.422, 0.067, 0.041, 0.07, 0.02815, 0.078361, 1.1325, 0

## Spectral Type and Teff to estimate Luminosity:

Use Table from Pecaut & Mamajek (2013, ApJS, 208, 9; http://adsabs.harvard.edu/abs/2013ApJS..208....9P) accessed from https://www.pas.rochester.edu/~emamajek/EEM_dwarf_UBVIJHK_colors_Teff.txt

In [5]:
x = pd.read_table('EEM_dwarf_UBVIJHK_colors_Teff.txt', delim_whitespace=True, comment='#',
                 nrows=118)

# Turn spectral types into numbers to aid in plotting:
def GetSpTNumber(j):
    sptletters = ['O','B','A','F','G','K','M','L','T','Y']
    sptnumbers = [0,1,2,3,4,5,6,7,8,9]
    ind = np.where([x['SpT'][j][0]==sptletters[i] for i in range(len(sptletters))])
    n = sptnumbers[ind[0][0]]
    n += float(x['SpT'][j][1])/10
    if '.' in x['SpT'][j]:
        n += 0.05
    return n

x['SpT Numbers'] = np.nan
for j in range(len(x)):
    x['SpT Numbers'][j] = GetSpTNumber(j)

In [6]:
# Generate interpolation splines:
from scipy.interpolate import UnivariateSpline
GetTeffSpl = UnivariateSpline(x['SpT Numbers'], x['Teff'])
GetSpTSpl = UnivariateSpline(x['Teff'][::-1],x['SpT Numbers'][::-1])

# Turn SpT string into numbers in the planets table:
def GetSpTNumber(j):
    sptletters = ['O','B','A','F','G','K','M','L','T','Y']
    sptnumbers = [0,1,2,3,4,5,6,7,8,9]
    ind = np.where([p['st_spectype'][j][0]==sptletters[i] for i in range(len(sptletters))])
    try:
        n = sptnumbers[ind[0][0]]
    except IndexError:
        pass
    try:
        n += float(p['st_spectype'][j][1])/10
        if '.5' in p['st_spectype'][j]:
            n += 0.05
        return n
    except:
        pass


p['SpT Number'] = np.nan
for j in range(len(p)):
    try:
        np.isnan(p['st_spectype'][j])
        pass
    except TypeError:
        p['SpT Number'][j] = GetSpTNumber(j)
        
p['SpT Number'][322] = 6.3
p['SpT Number'][324] = 6.6

#### If it doesn't have a spectral type infer it from Mamjek table and Teff

In [7]:
p['Mam Inferred SpT from Teff'] = np.nan

for j in range(len(p)):
    try:
        if np.isnan(p['st_spectype'][j]):
            if not np.isnan(p['st_teff'][j]):
                p['Mam Inferred SpT from Teff'][j] = GetSpTSpl(p['st_teff'][j])  
    except TypeError:
        pass

#### If it doesn't have a Teff use Mamjek table to estimate:


In [8]:
p['Mam Inferred Teff from SpT'] = np.nan

for j in range(len(p)):
    try:
        if np.isnan(p['st_teff'][j]):
            if not np.isnan(p['SpT Number'][j]):
                p['Mam Inferred Teff from SpT'][j] = GetTeffSpl(p['SpT Number'][j])  
    except TypeError:
        pass

# Ok now I have radius, sep, teff, spt estimates

I'm going to do a quick cut on the ones really close to their star.  Converting the semi-major axis into angular units and removing the ones that at <1 lambda/D in separation at i' for GMT (800 nm at 25.4m primary)

In [9]:
p['pl_orbsmax'] # au
p['Sma arcsec'] = np.nan
for i in range(len(p)):
    p['Sma arcsec'][i] = p['pl_orbsmax'][i] / p['sy_dist'][i]
    
lod = 0.2063 * 0.8 / 25.4 #arcsec

p['Sma lod'] = np.nan
for i in range(len(p)):
    p['Sma lod'][i] = p['Sma arcsec'][i] / lod

In [10]:
pl = p.drop(np.where(p['Sma lod'] < 1)[0])
pl = pl.reset_index(drop=True)
print('N planets w Sep > 1 lod',len(pl))

spt = pl['SpT Number'].copy()
spt.loc[np.where(np.isnan(spt))[0]] = pl['Mam Inferred SpT from Teff'][np.where(np.isnan(spt))[0]]

pl = pl.drop(np.where(np.isnan(spt))[0])
pl = pl.reset_index(drop=True)
print('N of those with SpTs',len(pl))

rad = pl['pl_rade'].copy()
rad.loc[np.where(np.isnan(rad))[0]] = pl['M2R infered radius [Rearth]'][np.where(np.isnan(rad))[0]]

pl = pl.drop(np.where(np.isnan(rad))[0])
pl = pl.reset_index(drop=True)
print('N of those with radii',len(pl))

rad = pl['pl_rade'].copy()
rad.loc[np.where(np.isnan(rad))[0]] = pl['M2R infered radius [Rearth]'][np.where(np.isnan(rad))[0]]

spt = pl['SpT Number'].copy()
spt.loc[np.where(np.isnan(spt))[0]] = pl['Mam Inferred SpT from Teff'][np.where(np.isnan(spt))[0]]


N planets w Sep > 1 lod 697
N of those with SpTs 573
N of those with radii 544


### Consolidate a few things:

In [11]:
rad = pl['pl_rade'].copy()
rad.loc[np.where(np.isnan(rad))[0]] = pl['M2R infered radius [Rearth]'][np.where(np.isnan(rad))[0]]

spt = pl['SpT Number'].copy()
spt.loc[np.where(np.isnan(spt))[0]] = pl['Mam Inferred SpT from Teff'][np.where(np.isnan(spt))[0]]

teff = pl['st_teff'].copy()
ind = []
for i in range(len(pl)):
    if np.isnan(float(pl['st_teff'][i])):
        ind.append(i)
teff.loc[ind] = pl['Mam Inferred Teff from SpT'][ind]

pl['Re'] = rad
pl['SpT'] = spt
pl['StarTeff'] = teff
pl['StarTeff'][526] = float(pl['StarTeff'][526])
pl['StarTeff'][466] = float(pl['StarTeff'][466])

In [12]:
# For those stars without radius look up using Teff
x = pd.read_table('EEM_dwarf_UBVIJHK_colors_Teff.txt', delim_whitespace=True, comment='#',
                 nrows=118)

for i in range(len(x)):
    if x['R_Rsun'][i] == '...':
        x['R_Rsun'][i] = np.nan
    x['R_Rsun'][i] = float(x['R_Rsun'][i])

from scipy.interpolate import UnivariateSpline
GetStarRadSpl = UnivariateSpline(x['Teff'][:110][::-1], x['R_Rsun'][:110][::-1])
pl['Mam Inferred StarRad from Teff'] = np.nan

for j in range(len(pl)):
    try:
        if np.isnan(pl['st_rad'][j]):
            if not np.isnan(pl['st_teff'][j]):
                pl['Mam Inferred StarRad from Teff'][j] = GetStarRadSpl(pl['st_teff'][j])  
    except TypeError:
        pass

starrad = pl['st_rad'].copy()
ind = []
for i in range(len(pl)):
    if np.isnan(float(pl['st_rad'][i])):
        ind.append(i)
teff.loc[ind] = pl['Mam Inferred StarRad from Teff'][ind]

pl['StarRad'] = starrad

# Done, write it out:

In [13]:
pl.to_csv('GMagAO-X-target-list.csv',index=False)